# WS_ch08A.ipynb
# WESmith 04/16/23
## WS created this notebook to follow along chapter 8 code from book
## 'Bioinformatics with Python Cookbook' by Tiago Antao
## Each recipe will have its own notebook, suffixed by A, B, etc.¶
## also see book code in Chapter08/Data_Formats.py

In [14]:
from collections import defaultdict
import requests
from Bio import ExPASy, SwissProt
import pandas as pd
import io

In [110]:
# this chapter is out of date for accessing UNIPROT; see REST API and new formats at 
# https://www.uniprot.org/help/api_queries

In [111]:
# WS get the allowed fields from https://www.uniprot.org/help/return_fields
# WS this is the REST API
server = "https://rest.uniprot.org/uniprotkb/search?"

In [141]:
#query = 'organism_id:9606'  # test case: human id
query  = 'gene:P53'  # example in book
query += '+AND+reviewed:true'
params = {'query':query, 
          'format':'tsv',
          'fields':'accession,id,length,organism_id,organism_name,xref_pdb,xref_hgnc',
          'size':'50'} # size tested on 'organism_id' query; it works

In [143]:
def get_request(server, **kwargs):  # WS new function
    txt = server
    for j, k in kwargs.items():
        txt += j + '=' + k + '&'
    txt = txt[:-1] # strip off last '&'
    req = requests.get(txt)
    if not req.ok:
        req.raise_for_status()
    return req

In [144]:
req = get_request(server, **params)

In [147]:
uniprot_list = pd.read_table(io.StringIO(req.text))
uniprot_list.rename(columns={'Organism (ID)': 'ID'}, inplace=True)
#uniprot_list

In [148]:
p53_human = uniprot_list[
            (uniprot_list.ID == 9606) &
            (uniprot_list['Entry Name'].str.contains('P53'))]['Entry'].iloc[0]
p53_human

'P04637'

In [129]:
# next: retrieve the SwissProt record for the human P53